In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.data import Field
from torchtext.legacy.data import TabularDataset

from konlpy.tag import Hannanum
import pandas as pd

from nltk.tokenize import TreebankWordTokenizer
import nltk

from torchtext.legacy.data import TabularDataset

In [2]:
KOR = Field(sequential = True,
            use_vocab=True,
            tokenize=Hannanum,
            batch_first=True)

ENG = Field(sequential = True,
            use_vocab=True,
            lower=True,
            tokenize='spacy',
            batch_first=True)

C:\Users\kmiii\.conda\envs\py37\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [3]:
data = TabularDataset.splits(path='.', train='train_data.csv', valid='val_data.csv', test='test_data.csv', format='csv', fields=[("KOR", KOR), ('ENG', ENG)], skip_header=False)

OSError: [WinError 126] JVM DLL not found: ﻿KOR


In [ ]:
class Encoder(nn.Module):
    def __init__(self, n_tokens, n_inputs, n_hiddens, padding_idx, batch_first = True):
        super().__init__()
        self.n_hiddens = n_hiddens
        self.batch_first = batch_first
        self.embedding = nn.Embedding(n_tokens, n_inputs, padding_idx = padding_idx)
        self.bidirectional_gru = nn.GRU(n_inputs, n_hiddens, batch_first = batch_first, bidirectional=True)

    def init_hidden(self, batch_size):
        weight = next(self.parameters())
        h0 = weight.new_zeros(2, batch_size, self.n_hiddens)
        return h0

    def forward(self, x):
        input_length = torch.LongTensor([torch.max(x[i, :].data.nonzero()) + 1 for i in range(x.size(0))])
        hidden = self.init_hidden(x.size(0))
        x = self.embedding(x)
        x = nn.utils.rnn.pack_padded_sequence(x, input_length, batch_first = self.batch_first)
        output, hidden = self.bidirectional_gru(x, hidden)
        output = nn.utils.rnn.pad_packed_sequence(output, batch_first = self.batch_first)
        hidden = torch.cat([hidden[0], hidden[1]], dim = -1)
        return output, hidden



In [ ]:
class Decoder(nn.Module):
    def __init__(self, n_inputs, n_hiddens):
        super().__init__()
        self.

In [ ]:
encoder = Encoder(10,10, 10, 0)